### 1. Imports and Setup

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import kagglehub
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    Reshape, LSTM, GRU, Bidirectional, Input,
    Concatenate, Multiply
)
from tensorflow.keras.preprocessing import image
from transformers import ViTImageProcessor, TFViTForImageClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay, roc_auc_score, roc_curve
)
from sklearn.preprocessing import label_binarize

print("TensorFlow Version:", tf.__version__)

### 2. Data Downloading and Preparation

In [ ]:
try:
    path = kagglehub.dataset_download("marcozuppelli/stegoimagesdataset")
    print("Path to dataset files:", path)
    source_path = path
    destination_path = '/content/stegoimagesdataset'

    if os.path.exists(destination_path):
        shutil.rmtree(destination_path)
    os.makedirs(destination_path, exist_ok=True)

    for item in os.listdir(source_path):
        s = os.path.join(source_path, item)
        d = os.path.join(destination_path, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, dirs_exist_ok=True)
        else:
            shutil.copy2(s, d)
    print(f"Data copied to {destination_path}")
except Exception as e:
    print(f"Could not download from Kaggle Hub. Error: {e}")
    destination_path = '/content/stegoimagesdataset'
    if not os.path.exists(destination_path):
        raise FileNotFoundError("Dataset not found.")

In [ ]:
def segregate_data_by_payload(original_data_path, new_base_path):
    print("\nSegregating data by payload type...")
    for split in ['train', 'test', 'val']:
        stego_path = os.path.join(original_data_path, split, split, 'stego')
        clean_path = os.path.join(original_data_path, split, split, 'clean')
        new_split_path = os.path.join(new_base_path, split)

        if os.path.exists(stego_path):
            for img_name in os.listdir(stego_path):
                try:
                    payload_class = img_name.split("_")[2]
                    class_dir = os.path.join(new_split_path, payload_class)
                    os.makedirs(class_dir, exist_ok=True)
                    shutil.copy(os.path.join(stego_path, img_name), class_dir)
                except IndexError:
                    print(f"Skipping file with unexpected name format: {img_name}")

        if os.path.exists(clean_path):
            new_clean_path = os.path.join(new_split_path, 'clean')
            shutil.copytree(clean_path, new_clean_path, dirs_exist_ok=True)
    print("Data segregation complete.")

new_data_path = "new_data"
if os.path.exists(new_data_path):
    shutil.rmtree(new_data_path)
segregate_data_by_payload(destination_path, new_data_path)

In [ ]:
def combine_and_resplit(source_base_path, final_base_path, train_ratio=0.8, test_ratio=0.1):
    print("\nCombining and re-splitting data...")
    all_files_by_class = {}
    for class_name in os.listdir(os.path.join(source_base_path, 'train')):
        all_files_by_class[class_name] = []
        for split in ['train', 'test', 'val']:
            class_folder = os.path.join(source_base_path, split, class_name)
            if os.path.exists(class_folder):
                all_files_by_class[class_name].extend([os.path.join(class_folder, f) for f in os.listdir(class_folder)])

    if os.path.exists(final_base_path):
        shutil.rmtree(final_base_path)

    for class_name, files in all_files_by_class.items():
        train_files, temp_files = train_test_split(files, train_size=train_ratio, random_state=42)
        val_ratio = 1 - (test_ratio / (1 - train_ratio))
        val_files, test_files = train_test_split(temp_files, train_size=val_ratio, random_state=42)

        for split_name, file_list in [('train', train_files), ('val', val_files), ('test', test_files)]:
            dest_dir = os.path.join(final_base_path, split_name, class_name)
            os.makedirs(dest_dir, exist_ok=True)
            for f in file_list:
                shutil.copy(f, dest_dir)
    print("Data re-splitting complete.")

final_data_path = "final_payload_data"
combine_and_resplit(new_data_path, final_data_path)

In [ ]:
IMG_SIZE_GENERAL = 512
IMG_SIZE_VIT = 224
BATCH_SIZE = 32

print(f"\nLoading data for general models (CNN, RNNs) with image size {IMG_SIZE_GENERAL}x{IMG_SIZE_GENERAL}...")
train_dataset_gen = tf.keras.utils.image_dataset_from_directory(
    os.path.join(final_data_path, "train"),
    labels='inferred', label_mode='int',
    image_size=(IMG_SIZE_GENERAL, IMG_SIZE_GENERAL),
    batch_size=BATCH_SIZE, shuffle=True, seed=123
)
val_dataset_gen = tf.keras.utils.image_dataset_from_directory(
    os.path.join(final_data_path, "val"),
    labels='inferred', label_mode='int',
    image_size=(IMG_SIZE_GENERAL, IMG_SIZE_GENERAL),
    batch_size=BATCH_SIZE, shuffle=False, seed=123
)
test_dataset_gen = tf.keras.utils.image_dataset_from_directory(
    os.path.join(final_data_path, "test"),
    labels='inferred', label_mode='int',
    image_size=(IMG_SIZE_GENERAL, IMG_SIZE_GENERAL),
    batch_size=BATCH_SIZE, shuffle=False
)

print(f"\nLoading data for ViT model with image size {IMG_SIZE_VIT}x{IMG_SIZE_VIT}...")
train_dataset_vit = tf.keras.utils.image_dataset_from_directory(
    os.path.join(final_data_path, "train"),
    labels='inferred', label_mode='int',
    image_size=(IMG_SIZE_VIT, IMG_SIZE_VIT),
    batch_size=BATCH_SIZE, shuffle=True, seed=123
)
val_dataset_vit = tf.keras.utils.image_dataset_from_directory(
    os.path.join(final_data_path, "val"),
    labels='inferred', label_mode='int',
    image_size=(IMG_SIZE_VIT, IMG_SIZE_VIT),
    batch_size=BATCH_SIZE, shuffle=False, seed=123
)
test_dataset_vit = tf.keras.utils.image_dataset_from_directory(
    os.path.join(final_data_path, "test"),
    labels='inferred', label_mode='int',
    image_size=(IMG_SIZE_VIT, IMG_SIZE_VIT),
    batch_size=BATCH_SIZE, shuffle=False
)

class_names = train_dataset_gen.class_names
num_classes = len(class_names)
print(f"Found {num_classes} classes: {class_names}")

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

def prepare(ds, augment=False):
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds_gen = prepare(train_dataset_gen, augment=True)
val_ds_gen = prepare(val_dataset_gen)
test_ds_gen = prepare(test_dataset_gen)

train_ds_vit = prepare(train_dataset_vit, augment=True)
val_ds_vit = prepare(val_dataset_vit)
test_ds_vit = prepare(test_dataset_vit)

### 3. Individual Model Definitions

In [ ]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        tf.keras.layers.Rescaling(1./255, input_shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu', name='feature_layer'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ], name="CNN_Model")
    return model

def build_rnn_model(input_shape, num_classes, rnn_type='lstm'):
    features_per_timestep = input_shape[1] * input_shape[2]
    timesteps = input_shape[0]
    RNN_LAYER = LSTM if rnn_type.lower() == 'lstm' else GRU

    model = Sequential([
        Reshape((timesteps, features_per_timestep), input_shape=input_shape),
        Bidirectional(RNN_LAYER(128, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(RNN_LAYER(64, name='feature_layer')),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ], name=f"{rnn_type.upper()}_Model")
    return model

def build_vit_model(num_classes):
    model_name = "google/vit-base-patch16-224-in21k"
    model = TFViTForImageClassification.from_pretrained(
        model_name,
        num_labels=num_classes,
        ignore_mismatched_sizes=True
    )
    return model

In [ ]:
input_shape_gen = (IMG_SIZE_GENERAL, IMG_SIZE_GENERAL, 3)
cnn_model = build_cnn_model(input_shape_gen, num_classes)
lstm_model = build_rnn_model(input_shape_gen, num_classes, rnn_type='lstm')
gru_model = build_rnn_model(input_shape_gen, num_classes, rnn_type='gru')
vit_model = build_vit_model(num_classes)

print("--- CNN Model Summary ---")
cnn_model.summary()
print("\n--- LSTM Model Summary ---")
lstm_model.summary()
print("\n--- GRU Model Summary ---")
gru_model.summary()
print("\n--- ViT Model (structure) ---")
print(f"ViT model loaded: {vit_model.name}")

### 4. Model Training

In [ ]:
models = {
    "CNN": cnn_model,
    "LSTM": lstm_model,
    "GRU": gru_model,
    "ViT": vit_model
}

histories = {}
trained_models = {}

vit_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

for name, model in models.items():
    print(f"\n--- Training {name} Model ---")
    
    if name == 'ViT':
        def vit_prepare_fn(image, label):
            return {'pixel_values': image}, label

        train_ds_vit_prepared = train_ds_vit.map(vit_prepare_fn)
        val_ds_vit_prepared = val_ds_vit.map(vit_prepare_fn)
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                      loss=vit_loss,
                      metrics=['accuracy'])
        
        history = model.fit(
            train_ds_vit_prepared,
            validation_data=val_ds_vit_prepared,
            epochs=5,
            verbose=1
        )
    else:
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        history = model.fit(
            train_ds_gen,
            validation_data=val_ds_gen,
            epochs=10,
            verbose=1
        )
        
    histories[name] = history
    trained_models[name] = model
    print(f"--- {name} Model Training Complete ---")

### 5. Individual Model Evaluation

In [ ]:
def plot_roc_curve(y_true, y_pred_proba, num_classes, class_names, model_name):
    y_true_bin = label_binarize(y_true, classes=range(num_classes))
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        roc_auc[i] = np.trapz(tpr[i], fpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
    roc_auc["micro"] = np.trapz(tpr["micro"], fpr["micro"])

    plt.figure(figsize=(10, 8))
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'micro-average ROC curve (area = {roc_auc["micro"]:.2f})',
             color='deeppink', linestyle=':', linewidth=4)

    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], lw=2,
                 label=f'ROC curve of class {class_names[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Multi-class ROC Curve for {model_name}')
    plt.legend(loc="lower right")
    plt.show()

def evaluate_model(model, test_ds, model_name, is_vit=False):
    print(f"\n--- Evaluating {model_name} Model ---")
    test_ds_eval = test_ds
    if is_vit:
        test_ds_eval = test_ds.map(lambda x, y: ({'pixel_values': x}, y))

    y_true = np.concatenate([y for x, y in test_ds], axis=0)
    predictions = model.predict(test_ds_eval)
    
    y_pred_proba = predictions
    if is_vit:
        y_pred_proba = tf.nn.softmax(predictions.logits, axis=-1).numpy()
        
    y_pred = np.argmax(y_pred_proba, axis=1)

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix for {model_name}")
    plt.show()

    try:
        roc_auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        print(f"Weighted Average ROC-AUC Score: {roc_auc:.4f}")
        plot_roc_curve(y_true, y_pred_proba, num_classes, class_names, model_name)
    except Exception as e:
        print(f"Could not calculate ROC-AUC score: {e}")

In [ ]:
for name, model in trained_models.items():
    is_vit_model = (name == "ViT")
    current_test_ds = test_ds_vit if is_vit_model else test_ds_gen
    evaluate_model(model, current_test_ds, name, is_vit=is_vit_model)

### 6. Combination Technique 1: Feature Fusion

In [ ]:
print("\n--- Combination Technique 1: Feature Fusion ---")

feature_extractors = {}
feature_extractors['CNN'] = Model(
    inputs=cnn_model.input,
    outputs=cnn_model.get_layer('feature_layer').output,
    name="CNN_Feature_Extractor"
)
feature_extractors['LSTM'] = Model(
    inputs=lstm_model.input,
    outputs=lstm_model.get_layer('feature_layer').output,
    name="LSTM_Feature_Extractor"
)
feature_extractors['GRU'] = Model(
    inputs=gru_model.input,
    outputs=gru_model.get_layer('feature_layer').output,
    name="GRU_Feature_Extractor"
)
feature_extractors['ViT'] = Model(
    inputs=trained_models['ViT'].input,
    outputs=trained_models['ViT'].vit.pooler.output,
    name="ViT_Feature_Extractor"
)

In [ ]:
def extract_features_from_model(model, dataset, is_vit=False):
    features_list = []
    labels_list = []
    
    dataset_to_use = dataset
    if is_vit:
        dataset_to_use = dataset.map(lambda x, y: ({'pixel_values': x}, y))

    for data_batch, labels_batch in dataset_to_use:
        feats = model.predict(data_batch)
        features_list.append(feats)
        labels_list.append(labels_batch.numpy())
    
    return np.concatenate(features_list), np.concatenate(labels_list)

train_features, val_features, test_features = {}, {}, {}
train_labels, val_labels, test_labels = None, None, None

for name, extractor in feature_extractors.items():
    print(f"Extracting features using {name}...")
    is_vit = (name == 'ViT')
    
    current_train_ds = train_ds_vit if is_vit else train_ds_gen
    current_val_ds = val_ds_vit if is_vit else val_ds_gen
    current_test_ds = test_ds_vit if is_vit else test_ds_gen

    train_features[name], temp_train_labels = extract_features_from_model(extractor, current_train_ds, is_vit)
    val_features[name], temp_val_labels = extract_features_from_model(extractor, current_val_ds, is_vit)
    test_features[name], temp_test_labels = extract_features_from_model(extractor, current_test_ds, is_vit)

    if train_labels is None:
        train_labels = temp_train_labels
        val_labels = temp_val_labels
        test_labels = temp_test_labels

print("\nFeatures extracted for all models.")

In [ ]:
print("\n--- Concatenation Fusion ---")

def build_and_evaluate_concat_fusion(model_names, train_feats, val_feats, test_feats):
    print(f"\nFusing models: {', '.join(model_names)}")
    
    train_input = [train_features[name] for name in model_names]
    val_input = [val_features[name] for name in model_names]
    test_input = [test_features[name] for name in model_names]

    inputs = [Input(shape=train_features[name].shape[1:]) for name in model_names]
    concatenated = Concatenate()(inputs)
    
    x = Dense(256, activation='relu')(concatenated)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    fusion_model = Model(inputs=inputs, outputs=output)
    fusion_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    fusion_model.fit(train_input, train_labels,
                     validation_data=(val_input, val_labels),
                     epochs=20, batch_size=BATCH_SIZE, verbose=0)
    
    y_pred_proba = fusion_model.predict(test_input)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    print("Classification Report:")
    print(classification_report(test_labels, y_pred, target_names=class_names))
    roc_auc = roc_auc_score(test_labels, y_pred_proba, multi_class='ovr', average='weighted')
    print(f"Weighted Average ROC-AUC Score: {roc_auc:.4f}")

build_and_evaluate_concat_fusion(['CNN', 'ViT'], train_features, val_features, test_features)
build_and_evaluate_concat_fusion(['LSTM', 'ViT'], train_features, val_features, test_features)
build_and_evaluate_concat_fusion(['CNN', 'LSTM', 'ViT'], train_features, val_features, test_features)

In [ ]:
print("\n--- Element-wise Product Fusion ---")
PROJECTION_DIM = 128

def build_and_evaluate_product_fusion(model_names, train_feats, val_feats, test_feats):
    print(f"\nFusing models (Product): {', '.join(model_names)}")

    inputs = [Input(shape=train_features[name].shape[1:]) for name in model_names]
    projected_layers = [Dense(PROJECTION_DIM, activation='relu')(inp) for inp in inputs]
    
    multiplied = Multiply()(projected_layers)
    
    x = Dense(128, activation='relu')(multiplied)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    fusion_model = Model(inputs=inputs, outputs=output)
    fusion_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    train_input = [train_features[name] for name in model_names]
    val_input = [val_features[name] for name in model_names]
    test_input = [test_features[name] for name in model_names]

    fusion_model.fit(train_input, train_labels,
                     validation_data=(val_input, val_labels),
                     epochs=20, batch_size=BATCH_SIZE, verbose=0)
    
    y_pred_proba = fusion_model.predict(test_input)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    print("Classification Report:")
    print(classification_report(test_labels, y_pred, target_names=class_names))
    roc_auc = roc_auc_score(test_labels, y_pred_proba, multi_class='ovr', average='weighted')
    print(f"Weighted Average ROC-AUC Score: {roc_auc:.4f}")

build_and_evaluate_product_fusion(['CNN', 'ViT'], train_features, val_features, test_features)
build_and_evaluate_product_fusion(['LSTM', 'ViT'], train_features, val_features, test_features)

### 7. Combination Technique 2: Ensemble Voting

In [ ]:
print("\n--- Combination Technique 2: Ensemble Voting ---")

all_pred_probas = {}
for name, model in trained_models.items():
    print(f"Getting predictions from {name}...")
    if name == 'ViT':
        test_ds_vit_prepared = test_ds_vit.map(lambda x, y: ({'pixel_values': x}, y))
        predictions = model.predict(test_ds_vit_prepared)
        all_pred_probas[name] = tf.nn.softmax(predictions.logits, axis=-1).numpy()
    else:
        all_pred_probas[name] = model.predict(test_ds_gen)

y_true_ensemble = np.concatenate([y for x, y in test_ds_gen], axis=0)

In [ ]:
print("\n--- Hard Voting (Majority Voting) ---")
all_preds = [np.argmax(p, axis=1) for p in all_pred_probas.values()]
stacked_preds = np.stack(all_preds, axis=1)

hard_vote_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=stacked_preds)

print("Classification Report (Hard Voting):")
print(classification_report(y_true_ensemble, hard_vote_preds, target_names=class_names))

In [ ]:
print("\n--- Soft Voting (Averaging Probabilities) ---")
avg_proba = np.mean(list(all_pred_probas.values()), axis=0)
soft_vote_preds = np.argmax(avg_proba, axis=1)

print("Classification Report (Soft Voting):")
print(classification_report(y_true_ensemble, soft_vote_preds, target_names=class_names))

roc_auc_soft = roc_auc_score(y_true_ensemble, avg_proba, multi_class='ovr', average='weighted')
print(f"Weighted Average ROC-AUC Score (Soft Voting): {roc_auc_soft:.4f}")

plot_roc_curve(y_true_ensemble, avg_proba, num_classes, class_names, "Soft Voting Ensemble")